In [10]:
import requests, sys, json
 
server = "https://rest.ensembl.org"
ext = "/info/species?"
 
r = requests.get(server+ext, headers={ "Content-Type" : "text/xml"})
 
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
 
 

In [12]:
import requests, sys, json

server = "https://rest.ensembl.org"
ext = "/info/species?"

# Make the request with the correct header for JSON
r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

if not r.ok:
    r.raise_for_status()
    sys.exit()

# Extract the response as JSON
data = r.json()

# Save the JSON data to a file
with open('species_info.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

print("JSON file has been saved.")


JSON file has been saved.


<bound method Response.json of <Response [200]>>